# QAOA implementation with qutip and JAX NOT WORKING 

In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append('..')
import numpy as np

import jax
import jax.numpy as jnp

from jax import grad, value_and_grad
from jax.experimental import optimizers
from jax.scipy.linalg import expm

from qutip.operators import sigmax, qeye
from qutip.tensor import tensor

# from util import load_problem_instance

ModuleNotFoundError: No module named 'src.util'

In [ ]:
def plus_state(q):       
    return 1 / jnp.sqrt(2**q) * jnp.ones((2**q, 1))

In [ ]:
def cost_func(x, cost, p, q, s):
    gamma_list, beta_list = jnp.split(x, 2)
    s = variational_state(cost, p, q, s, gamma_list, beta_list)
    # print(cost.shape)
    # print(s)
    # print(jnp.matmul(cost, s).shape)
    return jnp.real(jnp.vdot(jnp.transpose(s), jnp.multiply(cost, s)))

In [ ]:
def loss_fn(x, cost, p, q, s):
    f = cost_func(x, cost, p, q, s)
    # print(f)
    return ((-4) - f)**2

In [ ]:
def variational_state(cost, p, q, s, gamma_list, beta_list):
    for gamma, beta in zip(gamma_list, beta_list):
        s = jnp.multiply(U_C(gamma, cost), s)
        s = jnp.matmul(U_B(beta, q), s)
    return s 

In [ ]:
def U_C(gamma, cost):
    return jnp.exp(-1j * gamma * cost)

In [ ]:
def U_B(beta, q):
    X = 0
    for i in range(q):
        local_pauli_x_operation = [sigmax() if i==j else qeye(sigmax().dims[0][0]) for j in range(q)]
        X += tensor(local_pauli_x_operation)
    X = jnp.array(X.full())
    return expm(- 1j * beta * X)

In [ ]:
def step(step, opt_state):
  value, grads = jax.value_and_grad(loss_fn)(get_params(opt_state), cost, p, q, s)
  opt_state = opt_update(step, grads, opt_state)
  return value, opt_state

## Gradient descent based optimization 

In [ ]:
cost = jnp.array([-0, -3, -2, -3, -3, -4, -3, -2, -2, -3, -4, -3, -3, -2, -3, -0]).reshape(-1,1)
# print(cost.shape)
# cost =jnp.array( np.diag(cost[:,0]) )

# cost = load_problem_instance('CVRP_2_11_qubits_only_constraints_3_cities_1_truck/')
# cost =jnp.array( np.diag(cost[:,0]) )

In [ ]:
p = 1
q = int(np.log2(cost.shape[0]))
s = plus_state(q)
x = jnp.array(np.random.rand(2*p))

## Time testing 
### 11 qubits
#### unoptimized U_C full matrix 
9.09 s ± 506 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#### optimized U_C vector 
3.27 s ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# %timeit cost_func(x, cost, p, q, s)

In [ ]:
learning_rate = 1e-1
params = jnp.array(np.random.rand(2*p)) 
num_steps = 2
epochs = 1

opt_init, opt_update, get_params = optimizers.adam(learning_rate)
opt_state = opt_init(params)

for epoch in range(epochs):
    for i in range(num_steps):
        value, opt_state = step(i, opt_state)

TypeError: dot_general requires contracting dimensions to have the same shape, got [1] and [16].

In [ ]:
f = cost_func(jnp.array(opt_state.packed_state[0][0]), cost, p, q, s)
print(f)

[[-0.23699164-3.8696158j]
 [-0.669193  -3.6987243j]
 [-0.51675093-3.7019782j]
 [-0.5581325 -3.6299508j]
 [-0.6691935 -3.6987247j]
 [-0.6841117 -3.7158506j]
 [-0.5581324 -3.6299515j]
 [-0.51675034-3.7019782j]
 [-0.5167508 -3.7019782j]
 [-0.5581329 -3.6299512j]
 [-0.68411136-3.715851j ]
 [-0.6691934 -3.6987252j]
 [-0.5581322 -3.6299515j]
 [-0.5167508 -3.7019782j]
 [-0.6691935 -3.6987247j]
 [-0.23699164-3.8696156j]]
-557.3751
